# Transfer Learning Example: Adapting a Pre-Trained Convolutional Neural Network for Custom Image Classification


Transfer learning involves taking a model that has been trained on one task and applying it to a different but related task. This technique is useful because it allows a model to leverage knowledge (features, weights, and biases) from the original task, which can often improve learning efficiency and prediction performance on the new task.


_**Our task in the following is to reuse a neural network that has been trained on a different image classification problem to solve another one.**_


## New Clasification Task & Dataset


We are going to use [**Oxford Flowers102**](https://paperswithcode.com/dataset/oxford-102-flower) dataset, which contains images of 102 different types of flowers. The size of the dataset is about 700 MB.


In [ ]:
import tensorflow_datasets

dataset, dataset_info = tensorflow_datasets.load(
    "oxford_flowers102",
    as_supervised=True,
    with_info=True,
    split=["train", "test", "validation"],
)

# Unpack the dataset
train_data, test_data, validation_data = dataset

Let's have a look into the images and their class labels.

In [ ]:
n_classes = 102

# Assuming 'class_names' is a list of flower names corresponding to label indices
class_names = dataset_info.features['label'].names

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

def display_images_with_labels(dataset, class_names, num_samples=5):
    """
    Display a grid of images and their labels.

    Parameters:
    - dataset: tf.data.Dataset - The dataset from which to take the samples.
    - class_names: list - A list of class names corresponding to the label indices.
    - num_samples: int - The number of samples to display.
    """
    plt.figure(figsize=(10, 2 * num_samples))  # Adjust figure size based on the number of samples

    # Take 'num_samples' samples from the dataset
    for i, (image, label) in enumerate(dataset.take(num_samples)):
        ax = plt.subplot(1, num_samples, i + 1)  # Create a subplot for each sample
        ax.imshow(image.numpy().astype("uint8"))  # Convert to uint8 type if necessary
        ax.set_title(f"Label: {class_names[label.numpy()]}")
        ax.axis("off")  # Hide the axes

    plt.tight_layout()  # Adjust layout so images are neatly displayed
    plt.show()


In [ ]:

display_images_with_labels(train_data, class_names, num_samples=5)

## Why Transfer Learning?

A quick internet search did not give us a pre-trained model for this particular flower classification problem. So we need to train our own. But do we need to train it from scratch? Training an image classifier, often a convolutional neural network, may take significant compute time. 

Can we do more with less? Yes, we can reuse most parts of a network that has already been trained on a large image dataset, and only retrain the final layer(s) for our specific task. This approach is a classic example of transfer learning.

Transfer learning works well for image classification problems because the early layers of a CNN learn to detect generic features like edges, textures, and patterns, which are relevant across various image domains. The later layers learn more specific features tailored to the task at hand. By leveraging a pre-trained model, we can benefit from the knowledge it has already acquired and adapt it to our specific problem with minimal retraining.

This allows us to achieve good performance with less data and computation, as we only need to train the last layer(s) instead of training the entire network from scratch.

<div align="center">
    <img src="img/Visualization-of-Transfer-Learning-4.jpg" alt="Visualization of Transfer Learning" style="max-width:100%; height:auto;"/>
    <br>
    <figcaption style="color: gray;">Source: <a href="https://www.researchgate.net/figure/Visualization-of-Transfer-Learning-4_fig5_331983081">A Study Review: Semantic segmentation with Deep Neural Networks</a></figcaption>
</div>

## Getting a Pretrained Model


_MobileNetV2_ is a good choice as a base model for transfer learning because it is lightweight, computationally efficient, and has been pre-trained on a large dataset.

It has been pre-trained on the ImageNet dataset, which contains over 1.4 million images across 1,000 classes. This diverse and large-scale dataset allows MobileNetV2 to learn a wide range of features that can be effectively transferred to other image classification tasks.

Its compact architecture makes it suitable for mobile and resource-constrained environments while still achieving good performance. By leveraging MobileNetV2's pre-learned features, we can quickly adapt it to our specific task with minimal re-training.

<div align="center">
    <img src="img/The-visual-structure-of-MobileNetV2-18860668.png" style="max-width:100%; height:auto;"/>
    <br>
    <figcaption style="color: gray;">MobileNetV2 architecture visualized (<a href="https://www.researchgate.net/figure/The-visual-structure-of-MobileNetV2_fig3_356354780">Source</a>)</figcaption>
</div>

In [ ]:
from tensorflow import keras

With the help of `keras` we obtain the pre-trained model. Note the following arguments:

- `input_shape=(224, 224, 3)`: Specifies the expected input shape of the images. Here, the images will be resized to 224x224 pixels with 3 color channels (RGB).
- `include_top=False`: Excludes the fully connected layer at the top of the network. This allows us to customize the final layer(s) for our specific classification task.
- `weights="imagenet"`: Initializes the model with pre-trained weights from the ImageNet dataset, enabling transfer learning.

In [ ]:
# Load MobileNetV2 without the top layer
base_model = keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
)

base_model.summary()

### Prepare Dataset for Training


The following code prepares the image data for training, validation, and testing:

- The preprocessing function resizes images and normalizes pixel values.
- The labels are converted to one-hot encoded format, since the network architecture expects this.
- The training, validation, and test data pipelines apply preprocessing, shuffle the training data, batch the data, and use prefetching to optimize data loading.
- These steps ensure the data is in the correct format and efficiently loaded during model training and evaluation.

In [ ]:
import tensorflow as tf

In [ ]:
def preprocess(image, label):
    # Resize and normalize the images
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, depth=n_classes)  # Convert labels to one-hot
    return image, label


# Apply preprocessing, shuffle, batch, and prefetch to optimize loading
train_data = (
    train_data
    .map(preprocess)
    .shuffle(1000)
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
validation_data = (
    validation_data
    .map(preprocess)
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
test_data = (
    test_data
    .map(preprocess)
    .batch(32)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
train_data

In [ ]:
validation_data

In [ ]:
test_data

In [ ]:
import tensorflow as tf

# Assuming `train_data` is your dataset
for images, labels in train_data.take(1):
    print("Label examples:", labels.numpy())
    print("Shape of labels:", labels.shape)
    print("Batch shape:", images.shape)

### Assemble Model for Transfer Learning


We now assemble the new model to be trained on flower classification. 

First, we "freeze" the weights in the base model - the convolutional part of the network - so they are unaffected by training. We want to reuse them.

In [ ]:
base_model.trainable = False  # Freeze the convolutional base

Our network needs a new classification head. The original model has an output layer for 1000 classes  by default - change it to the number of flower types.


In [ ]:
# Create a new model on top of the output of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(n_classes, activation='softmax')  # Output layer n_classes
])

Finally we compile the model to set it up for training. As performance metrics, we look at _accuracy_, _precision_ and _recall_.

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy", Precision(), Recall()],
)

### Training

Let's train the model. We can expect the training to be much faster since only the weights of the classification head need to be adjusted.

In [ ]:
n_epochs = 25

In [ ]:
# Fit the model on the training data
history = model.fit(
    train_data,
    epochs=n_epochs,  # You can adjust the number of epochs based on the training performance
    validation_data=validation_data,
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Convert the history.history dict to a pandas DataFrame
    history_df = pd.DataFrame(history.history)
    
    # Use the 'plot' method to plot the DataFrame, one column vs. the index
    history_df.plot(figsize=(10, 5))
    plt.grid(True)
    #plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
    plt.title('Training and Validation Loss and Accuracy')
    plt.show()


In [ ]:
plot_training_history(history)

### Evaluation

Let's see how well our new model does in terms of metrics...

In [ ]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_data)

# Calculate F1 Score
if test_precision + test_recall > 0:
    test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
else:
    test_f1_score = None  # Handle case where F1 score cannot be calculated

# Create a DataFrame
results_df = pd.DataFrame({
    "Metric": ["Loss", "Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [
        test_loss,
        test_accuracy,
        test_precision,
        test_recall,
        test_f1_score
    ]
})

results_df

... and let's have a look at a sample of the image set:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def display_sample_predictions(model, dataset, class_names):
    plt.figure(figsize=(12, 10))
    
    # Take a single batch from the dataset
    for images, labels in dataset.shuffle(100).take(1):
        predictions = model.predict(images)
        
        # Reverse the normalization, assuming the images were normalized to [0, 1]
        images = images * 255
        
        for i in range(9):  # Display the first 9 images and predictions
            ax = plt.subplot(3, 3, i + 1)
            # Clip the values to be in the range [0, 255] and convert to integers
            img = tf.clip_by_value(images[i], 0, 255).numpy().astype("uint8")
            
            plt.imshow(img)
            predicted_label = class_names[np.argmax(predictions[i])]
            
            # Check if labels are one-hot encoded or integers
            if labels.ndim == 2:  # One-hot encoded
                true_label_index = np.argmax(labels[i])
            else:  # Integer labels
                true_label_index = labels[i]
                
            true_label = class_names[true_label_index]
            
            plt.title(f"Predicted: {predicted_label},\n True: {true_label}")
            plt.axis("off")
    plt.show()

In [ ]:
# Assuming 'class_names' is a list of flower names corresponding to label indices
class_names = dataset_info.features['label'].names

# Display predictions
display_sample_predictions(model, test_data, class_names)

## Conclusion

This simple workflow is a demonstration of how we can obtain a pretty good image classifier for a new classification tasks with minimal training resources. Neural networks are mostly feature extractors, so learning makes it possible to reuse that knowledge.

### Saving the Model for Later Use

We are going to reuse this model. This is how to save it to disk.

In [ ]:
from pathlib import Path

model_dir = Path("../models/tmp")

# Assuming 'model' is your Keras model
model.save(model_dir / "flowers-classifier-tl.keras")


---

_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2024 [Christian Staudt](https://clstaudt.me), [Katharina Rasch](https://krasch.io)_
